In [6]:
import json
import numpy as np
import redis
from torchvision import datasets, transforms

# Initialize Redis client
redis_client = redis.Redis(host='host.docker.internal', port=6379, db=0, decode_responses=False)  # Modify host/port as needed

In [12]:
# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Stability trick
    return e_x / e_x.sum(axis=1, keepdims=True)

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

class Neuron:
    def __init__(self, weights, bias, activation, is_final_layer=False):
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer

    def forward(self, inputs):
        """Compute neuron output across all samples"""
        z = np.dot(inputs, self.weights) + self.bias  # (num_samples, 1)
        return z if self.is_final_layer else self.activation_func(z)

class Layer:
    def __init__(self, layer_id, neurons, is_final_layer=False):
        self.layer_id = layer_id
        self.neurons = neurons
        self.is_final_layer = is_final_layer

    def forward(self, input_key):
        """Retrieve inputs from Redis, process layer, and store outputs back in Redis"""
        input_data = redis_client.get(input_key)
        input_data = np.frombuffer(input_data, dtype=np.float32).reshape(-1, len(self.neurons[0].weights))  # Ensure 2D shape

        # Compute activations for all neurons
        outputs = np.stack([neuron.forward(input_data) for neuron in self.neurons], axis=1)  # (num_samples, num_neurons)

        # Apply softmax at final layer
        if self.is_final_layer:
            outputs = softmax(outputs)

        # Store output in Redis under a single key
        redis_client.set(self.layer_id, outputs.astype(np.float32).tobytes())

        return self.layer_id  # Return key to next layer

def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))

    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neurons = [
            Neuron(
                weights=np.array(node['weights']),
                bias=np.array(node['biases']),
                activation=node['activation'],
                is_final_layer=(i == len(sorted_layers) - 1)
            )
            for node in layer_info['nodes']
        ]
        layers.append(Layer(layer_id=layer_name, neurons=neurons, is_final_layer=(i == len(sorted_layers) - 1)))

    return layers

def forward_pass(layers, input_data):
    """Pass entire dataset through the network"""
    redis_client.flushdb()  # Clear Redis before processing

    # Store entire dataset as input in Redis
    redis_client.set("input_layer", input_data.astype(np.float32).tobytes())

    input_key = "input_layer"

    # Process each layer
    for layer in layers:
        input_key = layer.forward(input_key)  # Each layer reads & writes to Redis

    # Retrieve final output
    final_layer_key = layers[-1].layer_id
    final_outputs = redis_client.get(final_layer_key)
    final_outputs = np.frombuffer(final_outputs, dtype=np.float32).reshape(input_data.shape[0], -1)  # (num_samples, num_classes)

    return np.argmax(final_outputs, axis=1)  # Return predicted class for all samples

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Process entire dataset at once
all_images = np.array([mnist_test[i][0].view(-1).numpy() for i in range(len(mnist_test))])  # (10000, 784)
all_labels = np.array([mnist_test[i][1] for i in range(len(mnist_test))])  # (10000,)

# Forward pass
predictions = forward_pass(network, all_images)

# Compute accuracy
accuracy = np.mean(predictions == all_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 96.73%
